# Finite Difference Methods - Coefficient Calculations

>This notebook documents a method used to calculate the coefficients for a finite difference scheme of order $n$

## Description of Scheme

Consider the sketch shown below:

![Sketch of Discrete Domain](StencilSketch.png)

Here we have broken the continuous domain of some function $u$ into seven discrete points.  The points $x_j$ are a fixed distance $\delta$ apart and we denote the function $u$ being evaluated at the $j$th point $u_n$.  Imagine, we are interested in calculating the derivative of $u$ at the point $j=i$,that is, we seek $\frac{\partial u}{\partial x}\big|_{x=x_i}$.  The first step in accomplishing this is to write out the Taylor expansion centered at some arbitrary point $x=a$.

$$u(x) = u(x_i) + \frac{du}{dx}\bigg|_{a}\frac{(x-a)}{\delta} + \frac{d^2u}{dx^2}\bigg|_{a}\frac{(x-a)^2}{2!\delta^2} + \frac{d^3u}{dx^3}\bigg|_{a}\frac{(x-a)^3}{3!\delta^3} + ... + \frac{d^nu}{dx^n}\bigg|_{a}\frac{(x-a)^n}{n!\delta^n}$$

Clearly though, this equation is of limited use because we only know the _functional value_ at any given point; we do not know the derivative - that is what we wish to solve for.  We could procede by truncating the series for all terms with derivatives of order > 1 and then solve directly for the $\frac{du}{dx}\big
|_{a}$ term; however, this would give us only a second order approximation (since all truncated terms are of $x^2$ or greater).  It would be desirable to develop a more general method for approximating the derivative to any order.  

Let's proceed by rewriting the Taylor expansion replacing the derivative terms with a set of constants.

$$u(x) =\alpha_0 + \alpha_1(x-a) + \alpha_2(x-a)^2 + \alpha_3(x-a)^3 + ... + \alpha_n(x-a)^n$$

This equation states that $u(x)$ can be expressed as an $n$th order polynomial.  Let's assume that we wish to _approximate_ $u$ using an $k$th order polynomial by truncating all terms of order greater than $x^k$.  Hence, we obtain:

$$u(x) \approx \bar{u}(x) = \alpha_0 + \alpha_1(x-a) + \alpha_2(x-a)^2 + \alpha_3(x-a)^3 + ... + \alpha_k(x-a)^k$$

We can solve the constants using the following steps:
1. Select $r=k+1$ different points in the domain near $x=a$
2. For each selected point, substitute the corresponding pair $(x_j,u_j)$ into the polynomial
3. Solve the resulting system of equations to obtain each constant.

The goal here is to obtain a polynomial that locally approximates the actual function near the point $x=a$, and then use this polynomial to approximate the derivative of the function.  Specifically, we want to approximate the function _near_ $x=x_i$; so, for example if we wish to calculate a second order polynomial we would have 

$$\bar{u}(x)=\alpha_0 + \alpha_1(x-x_i)+\alpha_2(x-x_i)^2$$  
To find the constants, we could select the following points to plug into the equation:

$$\{(u_{i-2},x_{i-2}),(u_{i-1},x_{i-1}),(u_i,x_i)\}$$

Note that any deviation from $x_i$ can be expressed in terms of $\delta$. For instance, we can write $x_{i-1} = x_i - \delta$.  Thus, substituting in the three points expressed above, we obtain the following sytem of equations:

$$\begin{align} \bar{u}(x_{i-2}) &= \alpha_1 + \alpha_2(x_{i} - 2\delta - x_{i}) + \alpha_3(x_{i} - 2\delta - x_{i})^2 \\ \bar{u}(x_{i-1}) &= \alpha_1 + \alpha_2(x_{i} - \delta - x_{i})+\alpha_3(x_{i} - \delta - x_{i})^2 \\ \bar{u}(x_{i}) &= \alpha_1 + \alpha_2(x_{i} - x_{i})+\alpha_3(x_{i} - x_{i})^2 \end{align} $$

Which simplifies to:

$$\begin{align} \bar{u}_{i-2} &= \alpha_1 - 2\alpha_2\delta+4\alpha_3\delta^2 \\ \bar{u}_{i-1} &= \alpha_1 - \alpha_2\delta+\alpha_3\delta^2 \\ \bar{u}_i &= \alpha_1 \end{align}$$

Solving this system gives us the following results:

$$\begin{align} \alpha_0&=u_i \\ \alpha_1&= \frac{3u_i - 4u_{i-1} + u_{i-2}}{2\delta} \\
\alpha_2 &= \frac{u_i - 2u_{i-1} + u_{i-2}}{2\delta^2} \end{align}$$

Which we can substitute into $\bar{u}$ - note that the value of $x_i$ is irrelevent because it cancels out for all calculations, thus to simplify the math we take $x_i = 0$ to obtain:

$$\bar{u}(x) = \big(1+ \frac{3}{2\delta}x + \frac{1}{2\delta^2}x^2\big)u_i -\big(\frac{2}{\delta}x + \frac{1}{\delta^2}x^2\big)u_{i-1} + \big(\frac{1}{2\delta}x +\frac{1}{2\delta^2}x^2 \big)u_{i-2} $$

Thus, we have a second order polynomial that locally approximates $u(x)$ near $x=x_i$.  In order to use this polynomial to calculate the derivative, we want to find the function value at the "half points" of the stencil, that is, we seek $\big\{u_{i+\frac{1}{2}},u_{i-\frac{1}{2}},u_{i-\frac{3}{2}}\big\}$.  These are found just by plugging in the appropriate $x$ value into $\bar{u}$.  After some algebra, we obtain:

$$\boxed{\begin{align} u_{i+\frac{1}{2}} \approx \bar{u}\big(\tfrac{1}{2}\delta\big) &= \frac{15}{8}u_i - \frac{5}{4}u_{i-1} +\frac{3}{8}u_{i-2} \\ u_{i-\frac{1}{2}} \approx \bar{u}\big(-\tfrac{1}{2}\delta\big) &= \frac{3}{8}u_i + \frac{3}{4}u_{i-1} - \frac{1}{8}u_{i-2} \\ u_{i-\frac{3}{2}} \approx \bar{u}\big(-\tfrac{3}{2}\delta\big) &= -\frac{1}{8}u_i + \frac{3}{4}u_{i-1} + \frac{3}{8}u_{i-2}  \end{align}}$$


Clearly, this is a rather tedious process, so it would be of interest to automate the determination of the proper polynomial coefficients given the desired order $k$.  I performed the hand calculations through a third order polynomial ($O(4)$). The Results are tabulated below:

|**Order**|$u_{n}$|$k_s=0$|$k_s=1$|$k_s=2$|$k_s=3$|
|---------|-----|---------------|---------------|---------------|----------------|
|  **4**  |$u_{i-3}$|$\frac{5}{16}$ |$-\frac{1}{16}$|$\frac{1}{16}$ |$-\frac{5}{16}$ | 
|    -    |$u_{i-2}$|$\frac{15}{16}$|$\frac{9}{16}$ |$-\frac{5}{16}$|$\frac{21}{16}$ |
|    -    |$u_{i-1}$|$-\frac{5}{16}$|$\frac{9}{16}$ |$\frac{15}{16}$|$-\frac{35}{16}$|
|    -    |$u_{i}  $|$\frac{1}{16}$ |$-\frac{1}{16}$|$\frac{5}{16}$ |$\frac{35}{16}$ |
|  **3**  |$u_{i-2}$|$\frac{3}{8}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |
|    -    |$u_{i-1}$|$\frac{3}{4}$  |$\frac{3}{4}$  |$-\frac{5}{4}$ |
|    -    |$u_{i}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |$\frac{15}{8}$ |
|  **2**  |$u_{i-1}$|$\frac{1}{2}$  |$-\frac{1}{2}$ |
|    -    |$u_{i}$  |$\frac{1}{2}$  |$\frac{3}{2}$  |

|**Order**|**l**|$k_s=0$|$k_s=1$|$k_s=2$|$k_s=3$|
|---------|-----|---------------|---------------|---------------|----------------|
|  **4**  |**0**|$\frac{5}{16}$ |$-\frac{1}{16}$|$\frac{1}{16}$ |$-\frac{5}{16}$ | 
|    -    |**1**|$\frac{15}{16}$|$\frac{9}{16}$ |$-\frac{5}{16}$|$\frac{21}{16}$ |
|    -    |**2**|$-\frac{5}{16}$|$\frac{9}{16}$ |$\frac{15}{16}$|$-\frac{35}{16}$|
|    -    |**3**|$\frac{1}{16}$ |$-\frac{1}{16}$|$\frac{5}{16}$ |$\frac{35}{16}$ |
|  **3**  |**0**|$\frac{3}{8}$  |$-\frac{1}{8}$ |$\frac{3}{8}$  |
|    -    |**1**|$\frac{3}{4}$  |$\frac{3}{4}$  |$-\frac{5}{4}$ |
|    -    |**2**|$-\frac{1}{8}$ |$\frac{3}{8}$  |$\frac{15}{8}$ |
|  **2**  |**0**|$\frac{1}{2}$  |$-\frac{1}{2}$ |
|    -    |**1**|$\frac{1}{2}$  |$\frac{3}{2}$  |

## General Solution to Interpolation Coefficients

We seek a general method for calculating the coefficients for the interpolation polynomial $\bar{u}(x)$.  Let's consider the following function:
$$\Pi_i(x) = (x-x_0)(x-x_1)(x-x_2)...(x-x_{i-1})(x-x_{i+1})...(x-x_{r-2})(x-x_{r-1})$$ 

The zeros of this function are clearly $x \in \{x_0,x_1,x_2...x_{i-1},x_{i+1}...x_{r-2},x_{r-1}\}$ so every $x$ in the domain is a zero except for $x_i$. That is, $$\Pi_i(x)=0 \quad \forall \quad x \neq x_i$$

Ultimately, we seek a polynomial that is unity at $x_i$ and zero everywhere else. $\Pi_i(x)$ nearly fits this criteria except it is not necessarily unity at $x_i$.  This can be overcome by simply finding what $\Pi_i(x)$ evaluates to at $x=x_i$ and then dividing the entire function by this value to obtain:

$$P_i(x)=\frac{\Pi_i(x)}{\Pi_i(x_i)} = \frac{(x-x_0)(x-x_1)(x-x_2)...(x-x_{i-1})(x-x_{i+1})...(x-x_{r-2})(x-x_{r-1})}{(x_i-x_0)(x_i-x_1)(x_i-x_2)...(x_i-x_{i-1})(x_i-x_{i+1})...(x_i-x_{r-2})(x_i-x_{r-1})}$$

With this function, we can now define our polynomial interpolation of $u(x)$ as where $R$ is the order of our polynomial:

$$\bar{u}(x) = \sum_{j=0}^{j=R} P_j(x)u(x_j)$$

However, we are interested in creating a local approximation of $u(x)$ near $x_i$ that is of $O(r)$. So, we are going to end up devising a polynomial of _degree_ $R=r-1$ using $r$ points; specifically, we will be using $r$ points starting at $x_i$ and moving leftward along the discrete domain until we reach $x_{i-R}$. Thus, we have:

$$\bar{u}_i(x) = \sum_{j=i-R}^{j=i} P_j(x)u(x_j)$$

Which, for clarity I will rewrite as:

$$\bar{u}_i(x) = \sum_{q=0}^{q=r-1} P_{i-q}(x)u(x_{i-q})$$

So, for an $r$ order approximation, we construct a $r-1$ degree polynomial using $r-1$ points to _locally_ approximate the function at each point in the discrete domain. Again, the end goal of this process is to approximate derivatives at _each specific point_ in the domain where we denote the current point of interest as $x_i$.  Using the equation defined above, lets look at some terms of this appoximation.

__First Term - q=0__
$$\frac{(x-x_{i-1})(x-x_{i-2})(x-x_{i-3})...(x-x_{i-(r-2)})(x-x_{i-(r-1)})}{(x_i-x_{i-1})(x_i-x_{i-2})(x_{i}-x_{i-3})...(x_i-x_{i-(r-2)})(x_i-x_{i-(r-1)})}u_{i}$$

__Second Term - q=1__
$$\frac{(x-x_{i})(x-x_{i-2})(x-x_{i-3})...(x-x_{i-(r-2)})(x-x_{i-(r-1)})}{(x_{i-1}-x_{i})(x_{i-1}-x_{i-2})(x_{i-1}-x_{i-3})...(x_{i-1}-x_{i-(r-2)})(x_{i-1}-x_{i-(r-1)})}u_{i-1}$$

__Third Term - q=2__
$$\frac{(x-x_{i})(x-x_{i-1})(x-x_{i-3})...(x-x_{i-(r-2)})(x-x_{i-(r-1)})}{(x_{i-2}-x_{i})(x_{i-2}-x_{i-1})(x_{i-2}-x_{i-3})...(x_{i-2}-x_{i-(r-2)})(x_{i-2}-x_{i-(r-1)})}u_{i-2}$$

$$\vdots$$

__Final Term - q=R=r-1__
$$\frac{(x-x_{i})(x-x_{i-1})(x-x_{i-2})...(x-x_{i-(r-3)})(x-x_{i-(r-2)})}{(x_{i-(r-1)}-x_{i})(x_{i-(r-1)}-x_{i-1})(x_{i-(r-1)}-x_{i-3})...(x_{i-(r-1)}-x_{i-(r-3)})(x_{i-(r-1)}-x_{i-(r-2)})}u_{i-(r-1)}$$

So, in general we have the following formula:

$$\bar{u}(x) = \sum_{q=0}^{q=r-1} \frac{\prod_{p=0}^{p=r-1} (x-x_{i-p})}{\prod_{p=0}^{p=r-1} (x_{i-q}-x_{i-p})}u_{i-q} \qquad where \;p\neq q $$

Next, we simply by letting $x_i=0$ and(assuming a constant step size) $x_{i-q} = -\delta q$.  Additionally, since our derivative calcualtion scheme requires we evaluate this polynomial at the half grid points (e.g. at $x_{i+\frac{1}{2}}$,$x_{i-\frac{1}{2}}$... ) we can let $x=(\frac{1}{2}-k)\delta$ where $k$ is particular stencil we are interested in. So, for the $q$ coefficient of the $r$ order approximation _evaluated_ at $x=x_i + (\frac{1}{2}-k)\delta$, we have:

$$ a_{r,k,q}= \frac{\prod_{p=0}^{p=r-1} ([\frac{1}{2}-k]\delta-(-\delta p))}{\prod_{p=0}^{p=r-1} ((-\delta q)-(-\delta p))} \qquad where \;p\neq q \quad  and \quad q \in [0,r-1]$$

which can be simplified by factoring out the $\delta$:

$$ a_{r,k,q}= \frac{\prod_{p=0}^{p=r-1} (\frac{1}{2}-k+ p)}{\prod_{p=0}^{p=r-1} (-q + p)} \qquad where \;p\neq q \quad and \quad q \in [0,r-1]$$

The denominator can be simplified by realizing that: 

$$\prod_{p=0}^{p=r-1} (-q + p) = (-1)^q \,(q!)\,(r-1 -q)! \quad where \quad q \in [0,r-1]$$


Using the above equation, we can solve for the interpolation coefficients for any order given a stencil.`b


We can approximate the derivative thusly:

$$\frac{\partial u}{\partial x}\bigg|_{x_i} \approx \frac{\bar{u}_()}{}$$

## Discussion with Dr. Shahbazi
---
As I had mentioned earlier, we have the case where we know a certain function on a domain $f(x)$ and we wish to differentiate this function over this interval.  There are many ways we could proceed, but our primary goal is to reconstruct a polynomial approximation of $f(x)$ locally and then use this reconstruction to evaluate the derivative.  We will call this polynomial approximation $P(x)$ - to find an $r$th order polynomial $P(x)$, we need $r+1$ points


Considering the error term, it is evident that a local reconstruction of the polynomial is preferred to a global reconstruction because we can adjust.


Interpolation gives larger error near boundaries. Because the termms grow larger and larger

We want our stencil to be as symetric as possible.

Reconstruction refers to rebuilding a continuous function given a dataset.

1. Find table of $a_{r,k_s,l}$ values for $r \in [0,10]$ and $K_s \in [0,10]$
2. Consider a smooth function $f(x)=sin(2\pi x)$ for $x\in [0,1]$. Approximate $\frac{df}{dx}$. Find and report the error for increasing order - refining gridpoint
3. Consider a smooth function that has a discontinuous or (nearly) discontinuous derivative - show error with increasing order.   


Report the $L2$ Norm for the error, that is if the absolute error vector is just the difference between our approximation and the actual value at each grid point.  The error is then just he magnitude of this vecgtor 


---


In [1]:
import numpy as np
import pandas as pd
import scipy.special as sp
#from fractions import Fraction as frac

In [2]:
def fd_coeffs(r,k):
    R=r-1
    q=np.arange(0,r,1)
    denominators=(-1)**q*sp.factorial(q)*sp.factorial(R-q)
    numerators=np.zeros(r)
    
    for p in q:
        numerators[p]=np.prod(1/2-k+q[(q!=p)])
        
    return numerators/denominators

In [4]:
fd_coeffs(15,0)

array([ 4.33393344e+00, -2.02250227e+01,  7.88775887e+01, -2.25364539e+02,
        4.82029708e+02, -7.88775887e+02,  1.00113863e+03, -9.91603972e+02,
        7.65576596e+02, -4.56659724e+02,  2.06584161e+02, -6.85892075e+01,
        1.57755177e+01, -2.24722475e+00,  1.49445981e-01])